# Flask Basics, Lesson 6!!!

## Flask_Login

This is the 2nd tool for User Authentication!

the *__flask_login__* library is an extremely useful tool for this process!!

![SegmentLocal](https://media0.giphy.com/media/iOjNaLUI4oBgc/giphy.gif "segment")

The best way to show you how to use it is a large example!!:

In [ ]:
# __init__.py
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_login import LoginManager

login_manager = LoginManager()

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mysecretkey'
basedir = os.path.abspath(os.path.dirname(__file__))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'+os.path(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
Migrate(app, db)

login_manager.init_app(app)
#Pass your application to the login manager!
login_manager.login_view = 'login'
#the app will have a view called 'login' and that will link this to our login manager 

Now we have to make our __Models!!__ which will include the User

In [ ]:
# models.py
from myproject import db, login_manager
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import UserMixin

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(user_id)
#Allows us to get the specific user to see their ID

class User(db.Model, UserMixin):
    #UserMixin has all the management features of logging in users and authorizing them, so we don't have to do that ourselves 
    
    __tablename__ = 'users'
    
    id = db.Column(db.Integer, primary_key = True)
    email = db.Column(db.String(64), unique = True, index = True)
    #passing 64 characters sets a string limit, so the email cannot be more than 64 characters
    #unique = True, makes sure that two users don't have the same email address
    usernmane = db.Column(db.String(64), unique = True, index = True)
    password_hash = db.Column(db.String(128))
    
    def __init__(self, email, username, password):
        self.email = email
        self.username = username
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
    

Now we make our __Forms!__ one for logging in and one for registering!

In [ ]:
#forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField 
from wtforms.validators import DataRequired, Email, EqualTo
# The validator Email will make sure what is entered is in email format, such as includes the @
# The validator EqualTo will make sure two things match, such as confirming password when registering 
from wtforms import ValidationError 

class LoginForm(FlaskForm):
    email = StringField('Email', validators = [DataRequired(), Email()])
    password = PasswordField('Password', validators = [DataRequired()])
    submit = SubmitField('Login')

class RegistrationForm(FlaskForm):
    email = StringField('Email', validators = [DataRequired(), Email()])
    username = StringField('Username', validators = [DataRequired()])
    password = PasswordField('Password', validators = [DataRequired(), EqualTo('pass_confirm', message = 'Passwords must match!')])
    # the message is what is presented if they are not equal 
    pass_confirm = PasswordField('Confirm Password', validators = [DataRequired()])
    submit = SubmitField('Register!')
    
    def check_email(self, field):
        if User.query.filter_by(email = field.data).first():
            raise ValidationError('Your email has already been registered!')
        #Checks to see if the email is already taken
    
    def check_username(self, field):
        if User.query.filter_by(username = field.data).first():
            raise ValidationError('Username is taken!')
        #Like the above function but with username